# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd # import Pandas library

Read the file and save it to "credit_scoring".

In [2]:
credit_scoring = pd.read_csv('data_credit_scoring.csv') # read the credit_scoring_eng.csv

Get the first 10 table strings.

In [3]:
credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,masters degree,0,married,0,F,employee,0,253875.639453,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,112080.014102,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,145885.952297,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,267628.550329,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,158616.077870,to have a wedding
5,0,-926.185831,27,masters degree,0,civil partnership,1,M,partner,0,255763.565419,purchase of the house
6,0,-2879.202052,43,masters degree,0,married,0,F,partner,0,240525.971920,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,135823.934197,education
8,2,-6929.865299,35,MASTERS DEGREE,0,civil partnership,1,F,employee,0,95856.832424,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,144425.938277,purchase of the house for my family


Look at the general information of "credit_scoring".

In [4]:
credit_scoring.info() # look at the general information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Use describe() method for more information.

Check the numeric values.

In [5]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Check the quantitative values.

In [6]:
credit_scoring.describe(include='object')

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,secondary education,married,F,employee,wedding ceremony
freq,13750,12380,14236,11119,797


Check the "education" column. It looks strange.

In [7]:
len(credit_scoring['education_id'].unique())

5

Check the NaN values.

In [8]:
credit_scoring.isnull().sum() # look at the NaN values

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Conclusion

We have a table with 21525 borrowers. And there is information in 12 columns:

- children : the number of children in the family
- days_employed: how long the customer has worked
- dob_years: the customer’s age
- education: the customer’s education level
- education_id: identifier for the customer’s education
- family_status: the customer’s marital status
- family_status_id: identifier for the customer’s marital status
- gender: the customer’s gender
- income_type: the customer’s income type
- debt: whether the client has ever defaulted on a loan
- total_income: monthly income
- purpose: reason for taking out a loan

There are some incorrect values:
- children : the number of children "-1" and "20"
- days_employed: negative and very big values, such as "401755.400475". Also there is the 2174 NaN values
- dob_years: "0" is impossible age
- education and education_id: there is 15 types in education and 5 in education_id
- gender: 3 unique, need to check
- total_income: there is the 2174 NaN values
  
Maybe NaN values, because borrower didn't provide it, the data lost by mistake or clients are unemployed.  
Before we start our global research we should to fix it.

### Step 2. Data preprocessing

We will process NaN and negative values, change the data type, check the data for duplicates, lemmatize and categorize data.

### Processing missing values

**Process the "children" column.**

In [9]:
credit_scoring['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

We have 47 values with "-1" and 76 values with "20". We replace them mean value.

In [10]:
# count mean for "children" column
child_mean = int(credit_scoring[(credit_scoring['children'] != 20) & (credit_scoring['children'] != -1)]['children'].mean())
child_mean

0

In [11]:
credit_scoring['children'] = credit_scoring['children'].replace([-1, 20], [child_mean, child_mean]) # replace them mean value

In [12]:
credit_scoring['children'].value_counts() # check the values

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Process the "days_employed" column.**

Look at the values > 0.

In [13]:
credit_scoring[credit_scoring['days_employed'] > 0]['days_employed'].sort_values()

20444    328728.720605
9328     328734.923996
17782    328771.341387
14783    328795.726728
7229     328827.345667
             ...      
7794     401663.850046
2156     401674.466633
7664     401675.093434
10006    401715.811749
6954     401755.400475
Name: days_employed, Length: 3445, dtype: float64

As we see min = 328728.720605, max = 401755.400475. It's impossible. Because 328728.720605 / 365 = 900 years. I think it's hours, not days.

Look at the values < 0.

In [14]:
credit_scoring[credit_scoring['days_employed'] < 0]['days_employed'].sort_values()

16335   -18388.949901
4299    -17615.563266
7329    -16593.472817
17838   -16264.699501
16825   -16119.687737
             ...     
2127       -34.701045
9683       -33.520665
6157       -30.195337
8336       -24.240695
17437      -24.141633
Name: days_employed, Length: 15906, dtype: float64

Convert the hours to days and negative values to positive.

In [15]:
# create the function to turn hours to days
def hours_to_days(days): 
    if days > 0:
        days = days / 24
    if days < 0:
        days *= -1
    return days
credit_scoring['days_employed'] = credit_scoring['days_employed'].apply(hours_to_days) # use apply() method to process column

Check the values.

In [16]:
credit_scoring['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

In [17]:
# count mean for "days_employed" column
days_employed_mean = credit_scoring['days_employed'].mean()
days_employed_mean

4641.641176180656

In [18]:
# replace NaN with mean in "days_employed"
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(days_employed_mean)

Divide by 365 "days_employed" and rename to "years_employed" for ease.

In [19]:
credit_scoring['days_employed'] = credit_scoring['days_employed'] / 365
credit_scoring = credit_scoring.rename(columns={"days_employed": "years_employed"})

**Process the "total_income" column.**

Look at the NaN values in the "income_type".

In [20]:
inc_type = credit_scoring.groupby('income_type')['total_income'] # group by "income_type"
inc_type.count().rsub(inc_type.size(), axis=0) # count the number of NaN in all of groups

income_type
civil servant                   147
employee                       1105
entrepreneur                      1
partner                         508
paternity / maternity leave       0
retiree                         413
student                           0
unempoyed                         0
Name: total_income, dtype: int64

We need to calculate the mean for employee, retiree, civil servant.

In [21]:
emp_mean = credit_scoring[credit_scoring['income_type'] == 'employee']['total_income'].mean() # employee mean
ret_mean = credit_scoring[credit_scoring['income_type'] == 'retiree']['total_income'].mean() # retiree mean
civ_mean = credit_scoring[credit_scoring['income_type'] == 'civil servant']['total_income'].mean() # civil servant

In [22]:
# replace NaN "total_income" values with mean of employee, partner, retiree, civil servant.
credit_scoring.loc[credit_scoring['total_income'].isnull() & (credit_scoring['income_type'] == 'employee'), 'total_income'] = emp_mean
credit_scoring.loc[credit_scoring['total_income'].isnull() & (credit_scoring['income_type'] == 'retiree'), 'total_income'] = ret_mean
credit_scoring.loc[credit_scoring['total_income'].isnull() & (credit_scoring['income_type'] == 'civil servant'), 'total_income'] = civ_mean

Fill in the 0's "partner" and "entrepreneur". Because "partner" hasn't any income and "entrepreneur" is only 1 value.

In [23]:
credit_scoring['total_income'] = credit_scoring['total_income'].fillna(0)

Check the "credit_scoring" for NaN again.

In [24]:
credit_scoring.isnull().sum() # use isnull() method for checking NaN values number

children            0
years_employed      0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

We've processed NaN and negative values and can move on.

**Process the "dob_years" column.**

Look at the "dob_years" column, where values < 18.

In [25]:
credit_scoring[credit_scoring['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

It's impossible values. Because you need to be adult to receive a loan. Replace these values with mean.

In [26]:
# count mean for "dob_years" column
dob_years_mean = int(credit_scoring['dob_years'].mean())
dob_years_mean

43

In [27]:
# replace "dob_years" values with mean
credit_scoring['dob_years'] = credit_scoring['dob_years'].replace(0, dob_years_mean)

In [28]:
credit_scoring[credit_scoring['dob_years'] < 18]['dob_years'].count() # check the values

0

We've got rid of 0's values

**Process the "gender" column.**

In [29]:
credit_scoring['gender'].value_counts() # check the values

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [30]:
credit_scoring[credit_scoring['gender'] == 'XNA'] # look at the row with "XNA"

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,6.461919,24,bachelor degree,2,civil partnership,1,XNA,partner,0,203905.157261,buy real estate


We'll replace it with "F". Because "income_type" is the "partner". It's most likely to be a woman.

In [31]:
credit_scoring[credit_scoring['gender'] == 'XNA'] = credit_scoring[credit_scoring['gender'] == 'XNA'].replace('XNA', 'F')

Check the values.

In [32]:
credit_scoring['gender'].value_counts() # check the values

F    14237
M     7288
Name: gender, dtype: int64

### Conclusion

We've processed NaN and negative values and can move on.

### Data type replacement

Replace a type for the "total_income" with "astype()" method. It's fast method. And check it.

In [33]:
credit_scoring['total_income'] = credit_scoring['total_income'].astype(int) # replace type with "astype()" method
credit_scoring['years_employed'] = credit_scoring['years_employed'].astype(int) # replace type with "astype()" method
credit_scoring.info() # check changes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
years_employed      21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [34]:
credit_scoring.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,masters degree,0,married,0,F,employee,0,253875,purchase of the house
1,1,11,36,secondary education,1,married,0,F,employee,0,112080,car purchase
2,0,15,33,Secondary Education,1,married,0,M,employee,0,145885,purchase of the house
3,3,11,32,secondary education,1,married,0,M,employee,0,267628,supplementary education
4,0,38,53,secondary education,1,civil partnership,1,F,retiree,0,158616,to have a wedding


### Conclusion

We've replaced the "float" with "int" for convenience and clarity.

### Processing duplicates

We will establish the presence of duplicates, delete and check them.

**Process the "education" and "education_id" column.**

In [35]:
credit_scoring['education'].value_counts() # check the values

secondary education    13750
masters degree          4718
SECONDARY EDUCATION      772
Secondary Education      711
bachelor degree          668
MASTERS DEGREE           274
Masters Degree           268
primary education        250
Bachelor Degree           47
BACHELOR DEGREE           29
PRIMARY EDUCATION         17
Primary Education         15
academic degree            4
ACADEMIC DEGREE            1
Academic Degree            1
Name: education, dtype: int64

We have duplicates in this column. Process them.

In [36]:
credit_scoring['education'] = credit_scoring['education'].str.lower() # use str.lower() method

In [37]:
credit_scoring['education'].value_counts() # check the values

secondary education    15233
masters degree          5260
bachelor degree          744
primary education        282
academic degree            6
Name: education, dtype: int64

So "education" and "education_id" are the same now.

**Delete all duplicates.**

In [38]:
credit_scoring.duplicated().sum() # summarizes duplicates

71

In [39]:
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True) # delete duplicates

In [40]:
credit_scoring.duplicated().sum() # check duplicates

0

In [41]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
years_employed      21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Conclusion

We've deleted all duplicates.

### Lemmatization

Look at the unique values.

In [42]:
credit_scoring['purpose'].value_counts() # use value_counts() method

wedding ceremony                                 791
having a wedding                                 768
to have a wedding                                765
real estate transactions                         675
buy commercial real estate                       661
housing transactions                             652
buying property for renting out                  651
transactions with the residential real estate    650
purchase of the house                            646
housing                                          646
purchase of the house for my family              638
construction of own property                     635
property                                         633
transactions with my real estate                 627
building a real estate                           624
buy real estate                                  621
purchase of my own house                         620
building a property                              619
property renovation                           

In [43]:
credit_scoring['purpose'].value_counts().sum() # count sum

21454

We will find lemma for values in the column "purpose".

Import nltk library and use WordNetLemmatizer.

In [44]:
# WordNetLemmatizer is imported as follows:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()

Make the lemma for the "purpose" column.

In [45]:
def lemma_text(text):
    words = text.split() # split text
    lemmas = ' '.join([wordnet_lemma.lemmatize(w, pos = 'n') for w in words])
    return lemmas

credit_scoring['purpose'] = credit_scoring['purpose'].apply(lemma_text) # use apply() method to process column

Check the results.

In [46]:
credit_scoring['purpose'].value_counts() # check the values

car                                             972
wedding ceremony                                791
having a wedding                                768
to have a wedding                               765
real estate transaction                         675
buy commercial real estate                      661
housing transaction                             652
buying property for renting out                 651
transaction with the residential real estate    650
purchase of the house                           646
housing                                         646
purchase of the house for my family             638
construction of own property                    635
property                                        633
transaction with my real estate                 627
building a real estate                          624
buy real estate                                 621
purchase of my own house                        620
building a property                             619
property ren

### Conclusion

We've done the lemmatization and we can use it when we want to group our data.

### Categorizing Data

**Categorize the data by "purpose" column.**

We have identified 4 groups in a "purpose" column and write the function for their isolation. Assign "car", "wedding", "real estate", "education".

In [47]:
def lemma_group(text):
    if 'car' in text:
        return 'car'
    if 'wedding' in text:
        return 'wedding'
    if 'estate' in text or 'property' in text or 'house' in text or 'housing' in text:
        return 'real estate'
    if 'university' in text or 'education' in text or 'educated' in text:
        return 'education'
    
credit_scoring['purpose'] = credit_scoring['purpose'].apply(lemma_group) # use apply() method to process column

In [48]:
credit_scoring['purpose'].value_counts().sum() # check the values

21454

Look at the our dataframe "credit_scoring".

In [49]:
credit_scoring.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,masters degree,0,married,0,F,employee,0,253875,real estate
1,1,11,36,secondary education,1,married,0,F,employee,0,112080,car
2,0,15,33,secondary education,1,married,0,M,employee,0,145885,real estate
3,3,11,32,secondary education,1,married,0,M,employee,0,267628,education
4,0,38,53,secondary education,1,civil partnership,1,F,retiree,0,158616,wedding


Which is the most popular purpose of loan?

In [50]:
credit_scoring['purpose'].value_counts()

real estate    10811
car             4306
education       4013
wedding         2324
Name: purpose, dtype: int64

As we see, the most popular type of loan is a "real estate". This is logical, because the cost of housing is high. People often cannot afford it without a loan. The most unpopular loan is a wedding.

**Categorize the data by "total_income".**

In [51]:
credit_scoring['total_income'].describe() # use describe() method to know min, 25 %, mean, 75 % and max income

count    2.145400e+04
mean     1.626814e+05
std      1.010722e+05
min      0.000000e+00
25%      1.032570e+05
50%      1.463810e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

Create 4 type of income.

In [52]:
def income_level(income):
    if income < 1.076522e+05:
        return 'poor'
    if income < 1.518870e+05:
        return 'middle'
    if income < 2.024170e+05:
        return 'wealthy'
    else:
        return 'rich'

credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_level) # use apply() method to process column
credit_scoring['income_level'].value_counts() # check the values

poor       5870
middle     5359
wealthy    5317
rich       4908
Name: income_level, dtype: int64

In [53]:
credit_scoring['income_level'].value_counts().sum() # check the values

21454

We divided people by income approximately equally.

**Categorize the data by "debt".**

In [54]:
credit_scoring['debt'].value_counts() # check the values

0    19713
1     1741
Name: debt, dtype: int64

Debt 1. The client has defaulted on a loan.  
Debt 0. The client repaid on a loan.

In [55]:
def debt_type(debt):
    if debt == 0:
        return 'on time'
    if debt == 1:
        return 'defaulted'

credit_scoring['debt_type'] = credit_scoring['debt'].apply(debt_type) # use apply() method to process column
credit_scoring['debt_type'].value_counts() # check the values

on time      19713
defaulted     1741
Name: debt_type, dtype: int64

In [56]:
1741 / 19713 * 100 # count the difference between "on time" and "defaulted"

8.831735403033532

We've made a more visual representation of debtors and on time paid. The difference between "on time" and "defaulted" is 8.83 %.  So the most pay on time.

**Categorize the data by "education".**

In [57]:
credit_scoring['education'].value_counts() # check the values

secondary education    15172
masters degree          5250
bachelor degree          744
primary education        282
academic degree            6
Name: education, dtype: int64

"secondary education" and "masters degree" are the most popular groups.

If we stand back and look, the results of "education" and "education_id" are the same.  
So we'll create dictionary for them.

In [58]:
# create the dictionary for 'education', 'education_id', drop duplicates
education_dict = credit_scoring[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,masters degree,0
1,secondary education,1
2,bachelor degree,2
3,primary education,3
4,academic degree,4


Then we can drop "education_id" column.

In [59]:
credit_scoring.drop(columns=['education_id'],inplace=True)

**Categorize the data by "family_status".**

If we stand back and look, the results of "family_status" and "family_status_id" are the same.
So we'll create dictionary for them.

In [60]:
# create the dictionary for 'family_status', 'family_status_id', drop duplicates
family_dict = credit_scoring[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
family_dict

,family_status,family_status_id
0,married,0
1,civil partnership,1
2,widow / widower,2
3,divorced,3
4,unmarried,4


Then we can drop "family_status_id" column.

In [61]:
credit_scoring.drop(columns=['family_status_id'],inplace=True)

In [62]:
credit_scoring.head() # check the results

,children,years_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,income_level,debt_type
0,1,23,42,masters degree,married,F,employee,0,253875,real estate,rich,on time
1,1,11,36,secondary education,married,F,employee,0,112080,car,middle,on time
2,0,15,33,secondary education,married,M,employee,0,145885,real estate,middle,on time
3,3,11,32,secondary education,married,M,employee,0,267628,education,rich,on time
4,0,38,53,secondary education,civil partnership,F,retiree,0,158616,wedding,wealthy,on time


### Conclusion

As we see, the most popular type of loan is a real estate.  
Most people pay on time.  
Most people who have received a loan with a college degree.

### Step 3. Answer these questions

**- Is there a relation between having kids and repaying a loan on time?**

Make the pivot table for analysis the "children" column.

In [63]:
# create the pivot table 
credit_child = credit_scoring.pivot_table(index=['children'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_child['debt_type_ratio'] = credit_child['defaulted'] / credit_child['on time'] * 100
# sort by "debt_type_ratio"
credit_child.sort_values(by='debt_type_ratio', ascending=True)

debt_type,defaulted,on time,debt_type_ratio
children,,,
0,1072.0,13142.0,8.157054
3,27.0,303.0,8.910891
1,444.0,4364.0,10.174152
2,194.0,1858.0,10.441335
4,4.0,37.0,10.810811
5,NaN,9.0,NaN


Count the difference.

In [64]:
dif_per_children = credit_child['debt_type_ratio'].max() - credit_child['debt_type_ratio'].min()
dif_per_children

2.6537570899159686

As we see, ratio in "debt_type_ratio" not that different. The difference is a insignificant (2.65 %).

Divide people into two groups with and without children.

Сheck borrowers, who have children. And see how many people paid the loan on time.

In [65]:
debtor_parent = credit_scoring[credit_scoring['children'] > 0]['debt_type'].value_counts(normalize=True) # use value_counts() method
debtor_parent

on time      0.907597
defaulted    0.092403
Name: debt_type, dtype: float64

90 % of people with children repaid the loan on time.

Сheck borrowers, who haven't children. And see how many people paid the loan on time.

In [66]:
debtor_childfree = credit_scoring[credit_scoring['children'] < 1]['debt_type'].value_counts(normalize=True) # use value_counts() method
debtor_childfree

on time      0.924581
defaulted    0.075419
Name: debt_type, dtype: float64

92 % of people without children repaid the loan on time.

Count the difference.

In [67]:
dif_per_child = (debtor_childfree.loc['on time'] - debtor_parent.loc['on time']) * 100
dif_per_child

1.6984713538204854

### Conclusion

So the difference between borrowers, who have children and who haven't is only **1.7 %**. I think it's not important difference to take it into account. So there isn't relation between having kids and repaying a loan on time.

**- Is there a relation between marital status and repaying a loan on time?**

Let's see which groups we have.

In [68]:
credit_scoring['family_status'].value_counts()

married              12339
civil partnership     4151
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

Make the pivot table for analysis the "family_status" column.

In [69]:
# create the pivot table 
credit_family = credit_scoring.pivot_table(index=['family_status'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_family['debt_type_ratio'] = credit_family['defaulted'] / credit_family['on time'] * 100
# sort by "debt_type_ratio"
credit_family.sort_values(by='debt_type_ratio', ascending=True)

debt_type,defaulted,on time,debt_type_ratio
family_status,,,
widow / widower,63,896,7.031250
divorced,85,1110,7.657658
married,931,11408,8.160940
civil partnership,388,3763,10.310922
unmarried,274,2536,10.804416


Count the difference.

In [70]:
dif_per_family = credit_family['debt_type_ratio'].max() - credit_family['debt_type_ratio'].min()
dif_per_family

3.7731664037854884

### Conclusion

As we see, ratio in "debt_type_ratio" not that different. The max difference is a insignificant (3.77 %). So there is no relation between marital status and repaying a loan on time.

**- Is there a relation between income level and repaying a loan on time?**

Make the pivot table for analysis the "income_level" colunm.

In [71]:
# create the pivot table 
credit_income = credit_scoring.pivot_table(index=['income_level'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_income['debt_type_ratio'] = credit_income['defaulted'] / credit_income['on time'] * 100
# sort by "debt_type_ratio"
credit_income.sort_values(by='debt_type_ratio', ascending=True)

debt_type,defaulted,on time,debt_type_ratio
income_level,,,
rich,345,4563,7.560815
poor,457,5413,8.442638
wealthy,464,4853,9.561096
middle,475,4884,9.725635


Count the difference.

In [72]:
dif_per_income = credit_income['debt_type_ratio'].max() - credit_income['debt_type_ratio'].min()
dif_per_income

2.1648194725117786

### Conclusion

As we see, ratio in "debt_type_ratio" not that different. The max difference is a insignificant (2.16 %). So there is no relation between income level and repaying a loan on time.

**- How do different loan purposes affect on-time repayment of the loan?**

Make the pivot table for analysis the "purpose_stemmed" column.

In [73]:
# create the pivot table 
credit_purpose = credit_scoring.pivot_table(index=['purpose'], columns='debt_type', values='debt', aggfunc='count')
# count the ratio defaulted / on time
credit_purpose['debt_type_ratio'] = credit_purpose['defaulted'] / credit_purpose['on time'] * 100
# sort by "debt_type_ratio"
credit_purpose.sort_values(by='debt_type_ratio', ascending=True)

debt_type,defaulted,on time,debt_type_ratio
purpose,,,
real estate,782,10029,7.797388
wedding,186,2138,8.699719
education,370,3643,10.156464
car,403,3903,10.325391


Count the difference.

In [74]:
dif_per_purpose = credit_purpose['debt_type_ratio'].max() - credit_purpose['debt_type_ratio'].min()
dif_per_purpose

2.5280031490537542

### Conclusion

As we see, ratio in "debt_type_ratio" not that different. The max difference is a insignificant (2.52 %). So loan purposes doesn't affect on-time repayment of the loan.

### Step 4. General conclusion

*Working hypothesis:*

Customer’s marital status and number of children has an impact on whether they will default on a loan.  

*Conclusion:*  

In conclusion, we can say that customer’s marital status and number of children hasn't an impact on whether they will default on a loan. The difference between the "defaulted" and "on time" values isn't significant. And doesn't exceed 3.77 % for marital status and 1.73 % for the number of children.  

And we do not see the connection between income, loan purposes and repayment of the loan on time.

Therefore, we can give loans more often not only to married, but also to other groups. And also to people with children. We need to look for ways to increase the return on credit in other categories.